In [1]:
from pathlib import Path
from jaal import Jaal
import pandas as pd
from bookgraph.reader import YaneuraBookReader
from bookgraph.util import get_project_root
from cshogi import Board

/workspaces/BookGraph/.venv/lib/python3.12/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
reader = YaneuraBookReader()

In [3]:
path = get_project_root() / Path("data/book.db")
nodes = list(reader.from_file(path))

from functools import reduce
from typing import Callable, Iterable
f: Callable[[list, list], list] = lambda li1, li2: li1 + li2
moves = reduce(f, (node.candidate_moves for node in nodes), []) #type: ignore

In [4]:
df = pd.DataFrame(nodes, columns=["node"])

In [5]:
df["board"] = df["node"].map(lambda x: x.sfen_for_hash)
df["next_move"] = df["node"].map(lambda x: x.candidate_moves[0].chosen_move_code)

In [6]:
df

,node,board,next_move
0,<bookgraph.book_tree.Node object at 0x7f7470e5...,+Bn1g1g1nl/2s2k1s1/pp1ppp1p1/6p1p/9/9/PP1PPPPP...,R*9b
1,<bookgraph.book_tree.Node object at 0x7f74707f...,+Bn1g1g1nl/2s2k1s1/pp1pppbp1/6p1p/9/9/PP1PPPPP...,3c9i+
2,<bookgraph.book_tree.Node object at 0x7f747083...,+Bn1g1g1nl/2sk2rs1/pppppp1pp/9/4b1p2/2P6/PP2PP...,3e3f
3,<bookgraph.book_tree.Node object at 0x7f747083...,+Bn1g1g1nl/2sk2rs1/pppppp1pp/9/4b4/2P3p2/PP2PP...,3f3g+
4,<bookgraph.book_tree.Node object at 0x7f747083...,+Bn1g1g1nl/2sk2rs1/pppppp1pp/9/4b4/2P3p2/PP2PP...,3i2h
...,...,...,...
130177,<bookgraph.book_tree.Node object at 0x7f746932...,lr4snl/3kg1gb1/p1nspp1pp/1ppp2p2/7P1/2PPP4/PPS...,9c9d
130178,<bookgraph.book_tree.Node object at 0x7f746932...,rn1g1g1nl/3s1k1b1/p2p3pp/1pp1Rpp2/9/2P6/PP1P1P...,B*4e
130179,<bookgraph.book_tree.Node object at 0x7f746932...,rn1g1g1nl/3s1k1b1/p2p3pp/1pp1spp2/9/2P6/PP1P1P...,5h5d
130180,<bookgraph.book_tree.Node object at 0x7f746932...,rn1gkg1nl/2s3rs1/pppppp1pp/6p2/7P1/2P3P2/PP1PP...,9a9b


In [7]:
moves = []
for moves_list in (node.candidate_moves for node in nodes):
    moves += moves_list
moves = moves[:100]

In [8]:
print(moves[0])

sfen_for_hash: +Bn1g1g1nl/2s2k1s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/9/+bNSGKGSNL b RLPrlp
next_sfen_for_hash: +Bn1g1g1nl/R1s2k1s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/9/+bNSGKGSNL w LPrlp
chosen_move_code: R*9b
expected_next_move_code: 4b3a
evaluation_value: 0
metadata: {'0': '32'}



In [9]:
board_set = set(move.sfen_for_hash for move in moves) | set(move.next_sfen_for_hash for move in moves)
sfen_to_id_dict = {sfen_for_hash: idx for idx, sfen_for_hash in enumerate(board_set)}

In [10]:
sfen_to_id_dict

{'1ns2g1nl/lkg2srb1/pppppp1pp/9/2P3p2/1R1P5/P1BSPPPPP/9/LN1GKGSNL w P': 0,
 '+Bn1g1g1nl/2sk3s1/ppppp2pp/9/6p2/2P2r1P1/PP1PPpP1P/1K5R1/LNSGSG1NL w LPb': 1,
 'kns2g1nl/l1g2srb1/pppppp1pp/9/1P4p2/2PPS4/P1B1PPPPP/1R3K3/LN1G1GSNL b -': 2,
 '1ns2g1nl/lkg3rb1/ppppsp1pp/4p4/PP4p2/2PP5/2BSPPPPP/1R5S1/LN1GKG1NL b -': 3,
 '1ns1g1snl/lkg3rb1/pppppp1pp/1P4p2/9/2PP5/P1BSPPPPP/1R7/LN1GKGSNL w -': 4,
 '1n1gkg1nl/r1s3rs1/pppppp1pp/6p2/7P1/2P3P2/PP1PPP2P/1B4GS1/LNSG2KNL w Lb': 5,
 '1ns1g1snl/lkg3rb1/p1pppp1pp/1R4p2/9/2PP5/P1BSPPPPP/9/LN1GKGSNL w Pp': 6,
 '1ns2g1nl/lkg2srb1/pppppp1pp/9/6p2/2PP5/P1BSPPPPP/1R4S2/LN1GKG1NL w P': 7,
 '1ns2g1nl/lkg2srb1/pppppp1pp/6p2/9/2PP1P3/PPBSP1PPP/1R2G1S2/LN2KG1NL w -': 8,
 '+Bn1g1gknl/R1s4s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/9/+bNSGKGSNL b LPrlp': 9,
 '1ns2g1nl/lkg3rb1/ppppps1pp/5p3/1PSP5/2P3p2/P1B1PPPPP/1R3K3/LN1G1GSNL b -': 10,
 '1npg1gknl/R+Bs4s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/2S6/+bN1GKGSNL w LPrl': 11,
 '1ns1g1snl/lkg3rb1/pppppp1pp/6p2/9/1RPP5/P1BSPPPPP/9/LN1GKGSNL w P': 

In [11]:
df_move = pd.DataFrame(moves, columns=["move"])

In [12]:
df_move["from"] = df_move["move"].map(lambda x: sfen_to_id_dict[x.sfen_for_hash])
df_move["to"] = df_move["move"].map(lambda x: sfen_to_id_dict[x.next_sfen_for_hash])

In [13]:
edge_df = df_move[["from", "to"]].copy()

In [14]:
node_df = pd.DataFrame(sfen_to_id_dict.keys(), columns=["title"], index=sfen_to_id_dict.values())

In [15]:
node_df["id"] = node_df.index

In [16]:
node_df

,title,id
0,1ns2g1nl/lkg2srb1/pppppp1pp/9/2P3p2/1R1P5/P1BS...,0
1,+Bn1g1g1nl/2sk3s1/ppppp2pp/9/6p2/2P2r1P1/PP1PP...,1
2,kns2g1nl/l1g2srb1/pppppp1pp/9/1P4p2/2PPS4/P1B1...,2
3,1ns2g1nl/lkg3rb1/ppppsp1pp/4p4/PP4p2/2PP5/2BSP...,3
4,1ns1g1snl/lkg3rb1/pppppp1pp/1P4p2/9/2PP5/P1BSP...,4
...,...,...
121,+Bn1g1g1nl/2s2k1s1/pp1ppp1p1/6p1p/9/9/PP1PPPPP...,121
122,1ns2g1nl/lk1g1rsb1/ppppp2pp/5pp2/9/4P1PP1/PP1P...,122
123,1ns2gs1l/lkg4b1/ppppppnpp/6r2/1P1P2p2/2P6/P1BS...,123
124,1ns2g1nl/lkg2srb1/pppppp1pp/9/1P7/2PP2p2/P1BSP...,124


In [18]:
node_df["board"] = node_df["title"].map(lambda x: Board(sfen=x))

In [31]:
node_df.iloc[0]["board"].to_svg().__str__()

'<svg xmlns="http://www.w3.org/2000/svg" version="1.1" xmlns:xlink="http://www.w3.org/1999/xlink" width="230.0" height="192.0" viewBox="0 0 230 192"><defs><g id="black-pawn"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#27497;</text></g><g id="black-lance"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#39321;</text></g><g id="black-knight"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#26690;</text></g><g id="black-silver"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#37504;</text></g><g id="black-gold"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#37329;</text></g><g id="black-bishop"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#35282;</text></g><g id="black-rook"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#39131;</text></g><g id="black

In [ ]:
Jaal(edge_df, node_df).plot()

Parsing the data...Done


/workspaces/BookGraph/.venv/lib/python3.12/site-packages/jaal/datasets/parse_dataframe.py:42: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['116' '83' '41' '70' '2' '12' '34' '85' '72' '21' '19' '71' '39' '92' '6'
 '42' '26' '77' '122' '117' '28' '103' '47' '125' '96' '105' '95' '53'
 '11' '58' '8' '124' '33' '59' '120' '55' '15' '46' '45' '45' '45' '108'
 '56' '57' '22' '27' '23' '115' '76' '88' '88' '73' '90' '36' '66' '89'
 '80' '50' '86' '109' '113' '13' '5' '64' '79' '74' '107' '107' '54' '51'
 '40' '111' '63' '32' '43' '43' '43' '61' '61' '38' '110' '106' '1' '37'
 '49' '44' '17' '101' '75' '62' '87' '30' '18' '81' '3' '93' '31' '20'
 '123' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/workspaces/BookGraph/.venv/lib/python3.12/site-packages/jaal/datasets/parse_dataframe.py:42: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise 

No trigger

Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



/workspaces/BookGraph/.venv/lib/python3.12/site-packages/IPython/core/completerlib.py:150: UserWarning:

This is now an optional IPython functionality, setting rootmodules_cache requires you to install the `pickleshare` library.

